<a href="https://colab.research.google.com/github/homerabbitsky/HelloAI/blob/main/METAL_Test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import time
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

gpu_device = torch.device("cuda")
tpu_device = torch.device("xla")
cpu_device = torch.device("cpu")

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

model = NeuralNetwork().to(cpu_device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
print("starting timer for training using CPU...")
start = time.time()
epochs = 30 
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, cpu_device)
print(f"completed training in ... {time.time()-start}s")

print("starting timer for testing using CPU...")
start = time.time()
test(test_dataloader, model, loss_fn, cpu_device)
print(f"completed testing in ... {time.time()-start}s")

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
starting timer for training using CPU...
Epoch 1
-------------------------------
loss: 2.293724  [   64/60000]
loss: 2.288591  [ 6464/60000]
loss: 2.260401  [12864/60000]
loss: 2.263943  [19264/60000]
loss: 2.251303  [25664/60000]
loss: 2.202639  [32064/60000]
loss: 2.224714  [38464/60000]
loss: 2.176765  [44864/60000]
loss: 2.175745  [51264/60000]
loss: 2.158056  [57664/60000]
Epoch 2
-------------------------------
loss: 2.155523  [   64/60000]
loss: 2.146245  [ 6464/60000]
loss: 2.082139  [12864/60000]
loss: 2.106647  [19264/60000]
loss: 2.046599  [25664/60000]
loss: 1.979487  [32064/60000]
loss: 2.010478  [38464/60000]
loss: 1.924026  [44864/60000]
loss: 1.937274  [51264/60000]
loss: 1.870329  [57664/60000]
Epoch 3
-------------------------------
loss: 1.902606  [   64/60000]
loss: 1.866295  [ 6464/60000]
loss: 1.748683  [12864/60000]
loss: 1.793863  [19264/60000]
loss: 1.666229  [25664/60